In [18]:
from music21 import *
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pickle
import glob

In [55]:




notes = []
duration = []

for midfile in glob.glob('midi_source/beethoven/sonata14/mond_3_format0.mid'):

    mid = converter.parse(midfile)

    notes_to_parse = None

    parts = instrument.partitionByInstrument(mid)

    if parts:  # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()

    else:  # file has notes in a flat structure
        notes_to_parse = mid.flat.notes
        print('teste')

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            a = str(element.pitch)
            
            #duration.append(str(element.duration.quarterLength))
            #a += ("|")
            #a += str(element.duration.quarterLength)
            
            notes.append(a)


        elif isinstance(element, chord.Chord):
            chord_pitches=[]


            for p in element.pitches:
                chord_pitches.append(p.midi)


            chord_pitches = str(chord_pitches).strip('[]')



            #chord_pitches += ("|")
            #chord_duration = str(element.duration.quarterLength)
            #chord_pitches += chord_duration
            
            chord_pitches += ("|chord")


            notes.append(chord_pitches)

            with open('data/sonata14/noduration/sonata14.3', 'wb') as path:
                pickle.dump(notes, path)


    print(notes)


['C#2', 'G#2', '49, 44|chord', 'E3', '56, 37|chord', 'C#3', '52, 44|chord', 'G#3', '61, 37|chord', 'E3', '56, 44|chord', 'C#4', '64, 37|chord', 'G#3', '61, 44|chord', 'E4', '68, 37|chord', 'C#4', '64, 44|chord', 'G#4', '73, 37|chord', 'E4', '68, 44|chord', 'C#5', '76, 37|chord', 'G#4', '73, 44|chord', 'E5', '68, 76, 80, 73, 49, 37|chord', '68, 73, 80, 76, 44|chord', 'C2', 'G#2', '48, 44|chord', 'E-3', '56, 36|chord', 'C3', '51, 44|chord', 'G#3', '60, 36|chord', 'E-3', '56, 44|chord', 'C4', '63, 36|chord', 'G#3', '60, 44|chord', 'E-4', '68, 36|chord', 'C4', '63, 44|chord', 'G#4', '72, 36|chord', 'E-4', '68, 44|chord', 'C5', '75, 36|chord', 'G#4', '72, 44|chord', 'E-5', '68, 72, 75, 80, 48, 36|chord', '72, 80, 68, 75, 44|chord', 'B1', 'C#3', '53, 44|chord', 'G#3', '61, 35|chord', 'F3', '56, 44|chord', 'C#4', '65, 35|chord', 'G#3', '61, 44|chord', 'F4', '68, 35|chord', 'C#4', '65, 44|chord', 'G#4', '73, 35|chord', 'F4', '68, 44|chord', 'C#5', '77, 35|chord', 'G#4', '73, 44|chord', 'F5', '

In [56]:
 '''with open('data/beethoven', 'rb') as path:
        notes = pickle.load(path)
     '''   
chars = sorted(list(set(notes)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(notes)
n_vocab = len(chars)
print ("Total Characters: ")
print(n_chars)
print ("Total Vocab: ")
print(n_vocab)


Total Characters: 
4309
Total Vocab: 
617


In [8]:
seq_length = 1000
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = notes[i:i + seq_length]
    seq_out = notes[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print( "Total Patterns: ") 
print(n_patterns)

Total Patterns: 
24897


In [5]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model

In [ ]:
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
# define the checkpoint
filepath="w{epoch:02d}-{loss:.4f}-moonlight.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=200, batch_size=100, callbacks=callbacks_list)

Epoch 1/200
5687/5687 [==============================] - 8s - loss: 5.9752     
Epoch 2/200
5687/5687 [==============================] - 7s - loss: 5.7115     
Epoch 3/200
5687/5687 [==============================] - 7s - loss: 5.6837     
Epoch 4/200
5687/5687 [==============================] - 7s - loss: 5.6705     
Epoch 6/200
5687/5687 [==============================] - 7s - loss: 5.6647     
Epoch 7/200
5687/5687 [==============================] - 7s - loss: 5.6622     
Epoch 8/200
5687/5687 [==============================] - 7s - loss: 5.6611     
Epoch 9/200
5687/5687 [==============================] - 7s - loss: 5.6568     
Epoch 10/200
5687/5687 [==============================] - 7s - loss: 5.6420     
Epoch 11/200
5687/5687 [==============================] - 7s - loss: 5.6238     
Epoch 12/200
5687/5687 [==============================] - 7s - loss: 5.5901     
Epoch 13/200
5687/5687 [==============================] - 7s - loss: 5.5420     
Epoch 14/200
5687/5687 [============

5687/5687 [==============================] - 7s - loss: 1.1925     
Epoch 71/200
5687/5687 [==============================] - 7s - loss: 1.1382     
Epoch 72/200
5687/5687 [==============================] - 7s - loss: 1.0955     
Epoch 73/200
5687/5687 [==============================] - 7s - loss: 1.0307     
Epoch 74/200
5687/5687 [==============================] - 7s - loss: 0.9998     
Epoch 75/200
5687/5687 [==============================] - 7s - loss: 0.9983     
Epoch 76/200
5687/5687 [==============================] - 7s - loss: 0.9782     
Epoch 77/200
5687/5687 [==============================] - 7s - loss: 0.9104     
Epoch 78/200
5687/5687 [==============================] - 7s - loss: 0.8654     
Epoch 79/200
5687/5687 [==============================] - 7s - loss: 0.8514     
Epoch 80/200
5687/5687 [==============================] - 7s - loss: 0.8375     
Epoch 81/200
5687/5687 [==============================] - 7s - loss: 0.9115     
Epoch 82/200
5687/5687 [=================

5687/5687 [==============================] - 7s - loss: 0.3266     
Epoch 107/200
5687/5687 [==============================] - 7s - loss: 0.3713     
Epoch 108/200
5687/5687 [==============================] - 7s - loss: 0.3755     
Epoch 109/200
5687/5687 [==============================] - 7s - loss: 0.3003     
Epoch 110/200
5687/5687 [==============================] - 7s - loss: 0.2846     
Epoch 111/200
5687/5687 [==============================] - 7s - loss: 0.2829     
Epoch 112/200
5687/5687 [==============================] - 7s - loss: 0.2856     
Epoch 113/200
5687/5687 [==============================] - 7s - loss: 0.2652     
Epoch 114/200
5687/5687 [==============================] - 7s - loss: 0.2730     
Epoch 115/200
5687/5687 [==============================] - 7s - loss: 0.2625     
Epoch 116/200
5687/5687 [==============================] - 7s - loss: 0.2259     
Epoch 117/200
5687/5687 [==============================] - 7s - loss: 0.2105     
Epoch 118/200
5687/5687 [=====

5687/5687 [==============================] - 7s - loss: 0.1354     
Epoch 146/200
5687/5687 [==============================] - 7s - loss: 0.1466     
Epoch 147/200
5687/5687 [==============================] - 7s - loss: 0.1294     
Epoch 148/200
5687/5687 [==============================] - 7s - loss: 0.1260     
Epoch 149/200
5687/5687 [==============================] - 7s - loss: 0.1304     
Epoch 150/200
5687/5687 [==============================] - 7s - loss: 0.1487     
Epoch 151/200
5687/5687 [==============================] - 7s - loss: 0.1503     
Epoch 152/200
5687/5687 [==============================] - 7s - loss: 0.1307     
Epoch 153/200
5687/5687 [==============================] - 7s - loss: 0.1216     
Epoch 154/200
5687/5687 [==============================] - 7s - loss: 0.6787     
Epoch 155/200
5687/5687 [==============================] - 7s - loss: 0.2370     
Epoch 156/200
5687/5687 [==============================] - 7s - loss: 0.1350     
Epoch 157/200
5687/5687 [=====

5687/5687 [==============================] - 7s - loss: 0.0750     
Epoch 187/200
5687/5687 [==============================] - 7s - loss: 0.0711     
Epoch 188/200
5687/5687 [==============================] - 7s - loss: 0.0791     
Epoch 189/200
5687/5687 [==============================] - 7s - loss: 0.0755     
Epoch 190/200
5687/5687 [==============================] - 7s - loss: 0.0800     
Epoch 191/200
5687/5687 [==============================] - 7s - loss: 0.0780     
Epoch 192/200
5687/5687 [==============================] - 7s - loss: 0.0688     
Epoch 193/200
5687/5687 [==============================] - 7s - loss: 0.0721     
Epoch 194/200
5687/5687 [==============================] - 7s - loss: 0.0801     
Epoch 195/200
5687/5687 [==============================] - 7s - loss: 0.0899     
Epoch 196/200
5687/5687 [==============================] - 7s - loss: 0.1947     
Epoch 197/200
5687/5687 [==============================] - 7s - loss: 0.1584     
Epoch 198/200
5687/5687 [=====